In [43]:
from keras.applications.mobilenet_v2 import MobileNetV2
import numpy as np
from imageio import imread
from keras.applications.mobilenet_v2 import preprocess_input
from skimage.transform import resize

In [44]:
model = MobileNetV2(weights="imagenet")
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [63]:
data = np.empty((24,224,224,3))
# print(data)
for i in range (12):
    im = imread('cats/f{:02d}.jpg'.format(i+1))
    im = preprocess_input(im)
    im = resize(im, output_shape=(224,224))
    data[i] = im
for i in range (12):
    im = imread('cats/m{:02d}.jpg'.format(i+1))
    im = preprocess_input(im)
    im = resize(im, output_shape=(224,224))
    data[i + 12] = im

labels = np.empty(24, dtype = int)
labels[:12] = 0 
labels[12:] = 1



C:\Users\marze\AppData\Local\Temp\ipykernel_7696\1328522048.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread('cats/f{:02d}.jpg'.format(i+1))
C:\Users\marze\AppData\Local\Temp\ipykernel_7696\1328522048.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread('cats/m{:02d}.jpg'.format(i+1))


In [64]:

from keras.applications.mobilenet_v2 import decode_predictions

predictions = model.predict(data)
for decoded_prediction in decode_predictions(predictions, top=1):
    for name, desc, score in decoded_prediction:
        print(' - {} ({:.2f}%)'.format(desc, 100 * score))
        


1/1 [==============================] - 1s 557ms/step
 - Egyptian_cat (50.55%)
 - tabby (64.92%)
 - tabby (46.92%)
 - Egyptian_cat (44.26%)
 - Persian_cat (92.98%)
 - lynx (32.72%)
 - refrigerator (22.00%)
 - Egyptian_cat (28.10%)
 - malamute (22.11%)
 - tabby (15.47%)
 - guillotine (25.38%)
 - lynx (32.76%)
 - Egyptian_cat (21.14%)
 - patas (12.75%)
 - Persian_cat (47.92%)
 - vizsla (24.20%)
 - Egyptian_cat (53.18%)
 - groenendael (13.37%)
 - tabby (17.44%)
 - axolotl (4.48%)
 - grey_fox (29.61%)
 - notebook (10.47%)
 - Egyptian_cat (8.67%)
 - Siamese_cat (45.47%)


In [65]:
from keras.layers import Dense 
from keras import Model

cat_output = Dense(2, activation='softmax')
cat_output = cat_output(model.layers[-2].output) #przedostatnia warstwa
cat_input = model.input
cat_model = Model(inputs=cat_input, outputs=cat_output)


for layer in cat_model.layers[:-1]:
    layer.trainable = False



In [66]:

cat_model.compile(
    loss ='sparse_categorical_crossentropy',
    optimizer = 'adam', 
    metrics = ['accuracy']

)

cat_model.fit(x=data, y=labels, epochs = 20, verbose =2) 

model.summary()


Epoch 1/20
1/1 - 1s - loss: 0.8821 - accuracy: 0.4167 - 1s/epoch - 1s/step
Epoch 2/20
1/1 - 0s - loss: 0.7501 - accuracy: 0.5417 - 167ms/epoch - 167ms/step
Epoch 3/20
1/1 - 0s - loss: 0.6342 - accuracy: 0.6667 - 162ms/epoch - 162ms/step
Epoch 4/20
1/1 - 0s - loss: 0.5340 - accuracy: 0.7917 - 166ms/epoch - 166ms/step
Epoch 5/20
1/1 - 0s - loss: 0.4489 - accuracy: 0.8333 - 166ms/epoch - 166ms/step
Epoch 6/20
1/1 - 0s - loss: 0.3777 - accuracy: 0.9167 - 168ms/epoch - 168ms/step
Epoch 7/20
1/1 - 0s - loss: 0.3186 - accuracy: 1.0000 - 171ms/epoch - 171ms/step
Epoch 8/20
1/1 - 0s - loss: 0.2698 - accuracy: 1.0000 - 170ms/epoch - 170ms/step
Epoch 9/20
1/1 - 0s - loss: 0.2296 - accuracy: 1.0000 - 165ms/epoch - 165ms/step
Epoch 10/20
1/1 - 0s - loss: 0.1964 - accuracy: 1.0000 - 172ms/epoch - 172ms/step
Epoch 11/20
1/1 - 0s - loss: 0.1688 - accuracy: 1.0000 - 173ms/epoch - 173ms/step
Epoch 12/20
1/1 - 0s - loss: 0.1457 - accuracy: 1.0000 - 172ms/epoch - 172ms/step
Epoch 13/20
1/1 - 0s - loss: 0.

In [67]:
predictions = cat_model.predict(data)

1/1 [==============================] - 1s 549ms/step


In [68]:
predictions.shape

(24, 2)

In [69]:
np.argmax(predictions, axis =1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int64)